# Notebook służący do ewaluacji
W nim chodzi o wygenerowanie X haseł na podstawie nauczonego modelu

In [1]:
%matplotlib widget
import numpy as np
import math
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import random

import matplotlib.pyplot as plt

import re

from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

from joblib import Parallel, delayed

import multiprocessing
from datetime import datetime

from torch.utils.data import DataLoader, Dataset
from IPython.display import clear_output

In [2]:
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

In [3]:
torch.manual_seed(1010101011)
random.seed(1010101011)

In [4]:
class CharacterLSTM(nn.Module):
    def __init__(self, vocabsize, lstmlayers, hiddensize):
        super(CharacterLSTM, self).__init__()
        
        ## WARSTWY
        self.embd = nn.Embedding(vocabsize, vocabsize)
        self.LSTM1 = nn.GRU(vocabsize, hiddensize, lstmlayers, batch_first=True, bidirectional=True)
        self.linear_ins = nn.Linear(2*hiddensize, vocabsize)

        self.drop = nn.Dropout(p=0.1)
        
        ## OUTS
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, hidden, NLL=True): 
        # WEJSCIE
        y0 = self.embd(x)
        
        # LSTM
        y, h1_ = self.LSTM1(y0, hidden)
        
        y = self.drop(y)
        
        # LINEAR OUT 1
        y = self.linear_ins(y)
        
        if NLL:
            y = self.softmax(y[:,-1])
        
        # zwrot
        return y, h1_

In [5]:
""" Odtworzenie zmiennych uczenia """
zmienne = torch.load("../models/zmienne_modelu.pth")

chartoidx = zmienne[0]

longestword = zmienne[1]
lstms = zmienne[2]
hiddensize = zmienne[3]

chlstm = torch.load("../models/NEWDS_START_bezrelu_lstm_15_hidden_40_cosine1e-8_rmsprop1e-7_50epoch_loss_2.423110246658325.pt", map_location=device).to(device)

## Działa dość wolno, słabo zoptymalizowane

In [28]:
chardict = list(chartoidx.keys())

""" parametr randomizacji """
""" im wyższa wartość tym bardziej randomowe i mniejsze odwzorowanie  """
temperature = 0.8


plikhasel = open("../haslaAI.txt", "a", encoding="utf8")

hiddens = torch.zeros(2*lstms, 1, hiddensize).to(device)

for _ in tqdm_notebook(range(140000)):
    with torch.no_grad():
        lastchar = 0
        cnt = 0

        chlstm.eval()
        znaki = [chartoidx["<START>"]]
        bazastart = len(znaki)

        for __ in range(longestword - bazastart+1):
            znaki.append(chartoidx["<EMPTY>"])

        for item in range(longestword - bazastart+1):
            x = torch.Tensor(znaki).long().to(device).view(1, -1)

            out, _ = chlstm(x, hiddens, NLL=False)
            
            zwrot = out[:,-1].view(-1)
            
            """ Rozkład wielomianowy """
            exped = zwrot.data.div(temperature).exp()
            top_i = torch.multinomial(exped, 1)
            charid = top_i[0]

            znaki[item+bazastart] = charid


        slowo = ""
        for item in znaki:
            if item != 0 and item != 1:
                slowo+=chardict[item]

#         print(slowo)
        plikhasel.write(slowo+"\n")
plikhasel.close()